## baseline_example

https://www.kaggle.com/ml100marathon/baseline-example

In [1]:
import os
import numpy as np
import pandas as pd
from datetime import date
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import KFold, train_test_split, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss, roc_auc_score, auc, roc_curve, accuracy_score
from sklearn.preprocessing import MinMaxScaler

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [2]:
# set data_path
data_path = '/Users/johnsonhuang/py_ds/ML_100/2nd-ML100Days/homework/Day_051To053_HW/'
column_description = pd.read_csv(os.path.join(data_path, 'column_description.csv'), encoding = "big5") # 因為預設不是utf8，如果不設這參數會unidecodeerror
column_description

,Column,Description
0,User_id,用戶 ID
1,Merchant_id,商家 ID
2,Coupon_id,優惠券 ID (null 代表無優惠券消費)
3,Discount_rate,"優惠券折價：[0,1] 代表折扣率；x:y 代表滿 x 減 y 元"
4,Distance,"用戶經常活動地點離商家最近距離 (x * 500 公尺), 0 表示低於 500 公尺, 1..."
5,Date_received,優惠券取得時間
6,Date,購買商品時間 (如果 Date is null & Coupon_id is not nul...


欄位說明 (與 column_description.csv 相同) 

<font color = "red">User_id</font>：用戶 ID 

<font color = "red">Merchant_id</font>：商家 ID 

<font color = "red">Coupon_id</font>：優惠券 ID (null 代表無優惠券消費) 

<font color = "red">Discount_rate</font>：優惠券折價</font>：[0,1] 代表折扣率；x:y 代表滿 x 減 y 元 

<font color = "red">Distance</font>：用戶經常活動地點離商家最近距離 (x * 500 公尺), 0 表示低於 500 公尺, 10 表示大於 5 公里。 

<font color = "red">Date_received</font>：優惠券取得時間。 

<font color = "red">Date</font>：購買商品時間 (如果 Date is null & Coupon_id is not null, 則該紀錄為有優惠券但未使用; 若為 Date is not null & Coupon_id is null, 則為普通消費日期; 若 Date is not null & Coupon_id is not null, 則表示優惠券消費日期)

## 第六版

In [3]:
# set data_path
data_path = '/Users/johnsonhuang/py_ds/ML_100/2nd-ML100Days/homework/Day_051To053_HW/'
train = pd.read_csv(os.path.join(data_path, 'train_offline.csv'))
test = pd.read_csv(os.path.join(data_path, 'test_offline.csv'))
test = test[~test.Coupon_id.isna()]
test.reset_index(drop=True, inplace=True)
print(train.shape)
print(test.shape)
train.head(5)

(1160742, 7)
(306313, 6)


,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0
1,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN
2,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN
3,1832624,3381,7610.0,200:20,0.0,20160429.0,NaN
4,2029232,3381,11951.0,200:20,1.0,20160129.0,NaN


In [4]:
set(train.columns) - set(test.columns)

{'Date'}

In [5]:
## Creat target label 
"""
According to the definition, 
1) buy with coupon within (include) 15 days ==> 1
2) buy with coupon but out of 15 days ==> 0
3) buy without coupon ==> -1 (we don't care)
"""
def label(row):
    # buy without coupon
    if np.isnan(row['Date_received']):
        return -1
    # buy with coupon within (include) 15 days ==> 1 ; buy with coupon but out of 15 days ==> 0
    if not np.isnan(row['Date']):
        td = pd.to_datetime(row['Date'], format='%Y%m%d') -  pd.to_datetime(row['Date_received'], format='%Y%m%d')
        if td <= pd.Timedelta(15, 'D'):
            return 1
    return 0

train["label"] = train.apply(label, axis=1)
train["label"].value_counts()

 0    710665
-1    413773
 1     36304
Name: label, dtype: int64

In [6]:
# Generate features - weekday acquired coupon
def getWeekday(row):
    if (np.isnan(row)) or (row==-1):
        return row
    else:
        return pd.to_datetime(row, format = "%Y%m%d").dayofweek+1 # add one to make it from 0~6 -> 1~7

train['weekday'] = train['Date_received'].apply(getWeekday)
test['weekday'] = test['Date_received'].apply(getWeekday)

# weekday_type (weekend = 1)
train['weekday_type'] = train['weekday'].astype('str').apply(lambda x : 1 if x in [6,7] else 0 ) # apply to trainset
test['weekday_type'] = test['weekday'].astype('str').apply(lambda x : 1 if x in [6,7] else 0 ) # apply to testset

# weekday get_dummies
weekdaycols = ['weekday_' + str(i) for i in range(1,8)]
print(weekdaycols)

tmpdf = pd.get_dummies(train['weekday'].replace(-1, np.nan))
tmpdf.columns = weekdaycols
train[weekdaycols] = tmpdf

tmpdf = pd.get_dummies(test['weekday'].replace(-1, np.nan))
tmpdf.columns = weekdaycols
test[weekdaycols] = tmpdf

['weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'weekday_7']


In [7]:
# Generate features - coupon discount and distance

"""
Discount_rate
ex. 20:1, 10:5, NaN
"""

def getDiscountType(row):
    if row == 'null':
        return 'null'
    elif ':' in row:
        return 1
    else:
        return 0

def convertRate(row):
    """Convert discount to rate"""
    if row == 'null':
        return 1.0
    elif ':' in row:
        rows = row.split(':')
        return 1.0 - float(rows[1])/float(rows[0])
    else:
        return float(row)

def getDiscountMan(row):
    if ':' in row:
        rows = row.split(':')
        return int(rows[0])
    else:
        return 0

def getDiscountJian(row):
    if ':' in row:
        rows = row.split(':')
        return int(rows[1])
    else:
        return 0

def processData(df):
    
    # convert discunt_rate
    df['discount_rate'] = df['Discount_rate'].astype('str').apply(convertRate)
    df['discount_man'] = df['Discount_rate'].astype('str').apply(getDiscountMan)
    df['discount_jian'] = df['Discount_rate'].astype('str').apply(getDiscountJian)
    df['discount_type'] = df['Discount_rate'].astype('str').apply(getDiscountType)
    
    # convert distance
    df.loc[df.Distance.isna(), "Distance"] = 99
    return df

train = processData(train)
test = processData(test)

train.head(5)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,label,weekday,weekday_type,...,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7,discount_rate,discount_man,discount_jian,discount_type
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0,-1,NaN,0,...,0,0,0,0,0,0,NaN,0,0,0
1,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN,0,3.0,0,...,0,1,0,0,0,0,0.95,20,1,1
2,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN,0,6.0,0,...,0,0,0,0,1,0,0.95,20,1,1
3,1832624,3381,7610.0,200:20,0.0,20160429.0,NaN,0,5.0,0,...,0,0,0,1,0,0,0.90,200,20,1
4,2029232,3381,11951.0,200:20,1.0,20160129.0,NaN,0,5.0,0,...,0,0,0,1,0,0,0.90,200,20,1


In [8]:
train.isnull().sum()

User_id               0
Merchant_id           0
Coupon_id        413773
Discount_rate    413773
Distance              0
Date_received    413773
Date             704033
label                 0
weekday          413773
weekday_type          0
weekday_1             0
weekday_2             0
weekday_3             0
weekday_4             0
weekday_5             0
weekday_6             0
weekday_7             0
discount_rate    413773
discount_man          0
discount_jian         0
discount_type         0
dtype: int64

In [9]:
# 去除掉 "buy without coupon ==> -1"
## Creat target label 
"""
According to the definition, 
1) buy with coupon within (include) 15 days ==> 1
2) buy with coupon but out of 15 days ==> 0
3) buy without coupon ==> -1 (we don't care)
"""
train = train[~train.label < 0]

In [10]:
# 在取得優惠卷15天以內或以外才消費的消費者，每人not unique商家數（購物趟數）
"""
g = df_preprocess_eventall.groupby('Member')
g.apply(lambda x: x[x['Status'] == 'WON']['Member_Win_Loss'].sum())
df1["overall_WON_amount"] = g.apply(lambda x: x[x['Status'] == 'WON']['Member_Win_Loss'].sum()).values   ＃用.values

sum(if(MEMBERWinLoss>0,MEMBERWinLoss,0))/(count(case Betstatus when 'WON' then 1 end)) as AvgWin,
"""

# train_merchantNumPerUserIn15D = train.groupby('User_id').apply(lambda x: x[x['label'] == 1]['Merchant_id'].count()).fillna(value=0).values   #用.values
# train_merchantNumPerUserMore15 = train.groupby('User_id').apply(lambda x: x[x['label'] == 0]['Merchant_id'].count()).fillna(value=0).values   #用.values


# train_merchantNumPerUserIn15D = (train.groupby('User_id').apply(lambda x: x[x['label'] == 1]['Merchant_id'].count())).to_frame().rename(columns = {0 : "merchantNumPerUserIn15D"}).reset_index()
# train_merchantNumPerUserMore15D = (train.groupby('User_id').apply(lambda x: x[x['label'] == 0]['Merchant_id'].count())).to_frame().rename(columns = {0 : "merchantNumPerUserMore15D"}).reset_index()
# train = pd.merge(train, train_merchantNumPerUserIn15D, how = "left", on = ["User_id"])
# train = pd.merge(train, train_merchantNumPerUserMore15D, how = "left", on = ["User_id"])

print("test 沒有 Date(消費日期) 這個欄位，就無法衍生出label這個欄位，所以不能照這個想法做")


test 沒有 Date(消費日期) 這個欄位，就無法衍生出label這個欄位，所以不能照這個想法做


In [11]:
# 在取得優惠卷15天以內或以外才消費的消費者，每人 平均購物距離

# train["distPerUser<=15"] = train.groupby('User_id').apply(lambda x: x[x['label'] == 1]['Distance'].mean()).fillna(value=0).values   #用.values
# train["distPerUser>15"] = train.groupby('User_id').apply(lambda x: x[x['label'] == 0]['Distance'].mean()).fillna(value=0).values   #用.values


# train_distPerUserIn15D = (train.groupby('User_id').apply(lambda x: x[x['label'] == 1]['Distance'].mean())).to_frame().rename(columns = {0 : "distPerUserIn15D"}).reset_index()
# train_distPerUserMore15D = (train.groupby('User_id').apply(lambda x: x[x['label'] == 0]['Distance'].mean())).to_frame().rename(columns = {0 : "distPerUserMore15D"}).reset_index()
# train = pd.merge(train, train_distPerUserIn15D, how = "left", on = ["User_id"])
# train = pd.merge(train, train_distPerUserMore15D, how = "left", on = ["User_id"])


In [12]:
# --- train ---

# 每人not unique商家數（購物趟數）
train_merchantNumPerUser = (train.groupby('User_id')['Merchant_id'].count()).to_frame().rename(columns = {"Merchant_id" : "merchantNumPerUser"}).reset_index()
train = pd.merge(train, train_merchantNumPerUser, how = "left", on = ["User_id"])

# 每人 平均購物距離
train_distPerUser = (train.groupby('User_id')['Distance'].mean()).to_frame().rename(columns = {"Distance" : "distPerUser"}).reset_index()
train = pd.merge(train, train_distPerUser, how = "left", on = ["User_id"])

# --- test ---

# 每人not unique商家數（購物趟數）
test_merchantNumPerUser = (test.groupby('User_id')['Merchant_id'].count()).to_frame().rename(columns = {"Merchant_id" : "merchantNumPerUser"}).reset_index()
test = pd.merge(test, test_merchantNumPerUser, how = "left", on = ["User_id"])

# 每人 平均購物距離
test_distPerUser = (test.groupby('User_id')['Distance'].mean()).to_frame().rename(columns = {"Distance" : "distPerUser"}).reset_index()
test = pd.merge(test, test_distPerUser, how = "left", on = ["User_id"])





In [13]:
# --- train ---

# 每人優惠卷數
train_couponNumPerUser = (train.groupby('User_id')['Coupon_id'].count()).to_frame().rename(columns = {"Coupon_id" : "couponNumPerUser"}).reset_index()
train = pd.merge(train, train_couponNumPerUser, how = "left", on = ["User_id"])

# 折扣率低於八折的優惠卷數量 (我覺得比較有優惠到的叫做八折啦)
train_goodDiscountCouNumPerUser = (train.groupby('User_id').apply(lambda x: x[x['discount_rate'] <= 0.8]['Coupon_id'].count())).to_frame().rename(columns = {0 : "goodDiscountCouNumPerUser"}).reset_index()
train = pd.merge(train, train_goodDiscountCouNumPerUser, how = "left", on = ["User_id"])

# --- test ---

# 每人優惠卷數
test_couponNumPerUser = (test.groupby('User_id')['Coupon_id'].count()).to_frame().rename(columns = {"Coupon_id" : "couponNumPerUser"}).reset_index()
test = pd.merge(test, test_couponNumPerUser, how = "left", on = ["User_id"])

# 折扣率低於八折的優惠卷數量 (我覺得比較有優惠到的叫做八折啦)
test_goodDiscountCouNumPerUser = (test.groupby('User_id').apply(lambda x: x[x['discount_rate'] <= 0.8]['Coupon_id'].count())).to_frame().rename(columns = {0 : "goodDiscountCouNumPerUser"}).reset_index()
test = pd.merge(test, test_goodDiscountCouNumPerUser, how = "left", on = ["User_id"])



In [14]:
# # 每個人 取得優惠券 卻沒消費的次數
# def wastedCouponNumPerUser(df) :
#     wastedCouponNumPerUser = (df.groupby('User_id').apply(lambda x: x[np.isnan(x['Date'])==True]['Coupon_id'].count())).to_frame().rename(columns = {"Coupon_id" : "wastedCouponNumPerUser"}).reset_index()
#     df = pd.merge(df, wastedCouponNumPerUser, how = "left", on = ["User_id"])
#     return df

# wastedCouponNumPerUser(train)
# wastedCouponNumPerUser(test)

print("test 沒有 Date(消費日期) 這個欄位，就無法衍生出label這個欄位，所以不能照這個想法做")

test 沒有 Date(消費日期) 這個欄位，就無法衍生出label這個欄位，所以不能照這個想法做


In [15]:
train.isnull().sum()

User_id                           0
Merchant_id                       0
Coupon_id                         0
Discount_rate                     0
Distance                          0
Date_received                     0
Date                         704033
label                             0
weekday                           0
weekday_type                      0
weekday_1                         0
weekday_2                         0
weekday_3                         0
weekday_4                         0
weekday_5                         0
weekday_6                         0
weekday_7                         0
discount_rate                     0
discount_man                      0
discount_jian                     0
discount_type                     0
merchantNumPerUser                0
distPerUser                       0
couponNumPerUser                  0
goodDiscountCouNumPerUser         0
dtype: int64

In [16]:
# 拿到優惠券低於八折的 每人 活躍地點距離商家的平均距離
def goodDiscountrateAvgDistPerUser(df) :
    goodDiscountrateAvgDistPerUser = (df.groupby('User_id').apply(lambda x: x[x['discount_rate'] <= 0.8]['Distance'].mean())).to_frame().rename(columns = {0 : "goodDiscountrateAvgDistPerUser"}).reset_index()
    df = pd.merge(df, goodDiscountrateAvgDistPerUser, how = "left", on = ["User_id"])
    return df

train = goodDiscountrateAvgDistPerUser(train)
test = goodDiscountrateAvgDistPerUser(test)

In [17]:
max(test["Distance"])

99.0

In [18]:
train.loc[train["Distance"]==99,:].iloc[0:5,:]

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,label,weekday,weekday_type,...,weekday_7,discount_rate,discount_man,discount_jian,discount_type,merchantNumPerUser,distPerUser,couponNumPerUser,goodDiscountCouNumPerUser,goodDiscountrateAvgDistPerUser
5,73611,2099,12034.0,100:10,99.0,20160207.0,NaN,0,7.0,0,...,1,0.900000,100,10,1,1,99.0,1,0,NaN
25,106443,450,3732.0,30:5,99.0,20160429.0,NaN,0,5.0,0,...,0,0.833333,30,5,1,2,49.5,2,0,NaN
28,2621792,4433,190.0,200:20,99.0,20160131.0,NaN,0,7.0,0,...,1,0.900000,200,20,1,1,99.0,1,0,NaN
54,983640,6134,767.0,200:20,99.0,20160124.0,NaN,0,7.0,0,...,1,0.900000,200,20,1,1,99.0,1,0,NaN
68,4465344,8386,9847.0,200:50,99.0,20160126.0,NaN,0,2.0,0,...,0,0.750000,200,50,1,7,99.0,7,1,99.0


In [19]:
# fill na of goodDiscountrateAvgDistPerUser 為 99 (max值)
train[['goodDiscountrateAvgDistPerUser']] = train[['goodDiscountrateAvgDistPerUser']].fillna(value=99)
test[['goodDiscountrateAvgDistPerUser']] = test[['goodDiscountrateAvgDistPerUser']].fillna(value=99)


In [20]:
# 每位 user 在各 merchant 拿到的 coupon 數
def couponNumInEachMerPerUser(df) :
    couponNumInEachMerPerUser = (df.groupby(['User_id', 'Merchant_id'])['Coupon_id'].count()).to_frame().rename(columns = {"Coupon_id" : "couponNumInEachMerPerUser"}).reset_index()
    df = pd.merge(df, couponNumInEachMerPerUser, how = "left", on = ["User_id", "Merchant_id"])
    return df

train = couponNumInEachMerPerUser(train)
test = couponNumInEachMerPerUser(test)

In [21]:
# 每位 user 有的 unique merchant 數
def NUniqueMerNumPerUser(df) :
    NUniqueMerNumPerUser = (df.groupby(['User_id'])['Merchant_id'].nunique()).to_frame().rename(columns = {"Merchant_id" : "NUniqueMerNumPerUser"}).reset_index()
    df = pd.merge(df, NUniqueMerNumPerUser, how = "left", on = ["User_id"])
    return df

train = NUniqueMerNumPerUser(train)
test = NUniqueMerNumPerUser(test)

In [22]:
## 看提交結果感覺是沒什麼幫助的特徵

# # 每位 user 在各 merchant 在 星期一到日 拿到的 coupon 數
# def day_i_couponNumInEachMerPerUser(df) :
# #     for i in range(1, 8) :
# #         exec(f'day_{i}_couponNumInEachMerPerUser = (df.groupby(["User_id", "Merchant_id"]).apply(lambda x: x[x["weekday_{i}"] == 1]["Coupon_id"].count())).to_frame().reset_index()')
# #         # rename column 用原本dict的方式那個大括號{}會跟exec的{}打架，所以改成這樣
# #         exec(f'day_{i}_couponNumInEachMerPerUser.columns = ["User_id", "Merchant_id", "day_{i}_couponNumInEachMerPerUser"]')
# #         exec(f'df = pd.merge(df, day_{i}_couponNumInEachMerPerUser, how = "left", on = ["User_id", "Merchant_id"])')
    
#     colFor_couponNumInEachMerPerUser = ["User_id", "Merchant_id", "Coupon_id", 
#                                         "weekday_1", "weekday_2", "weekday_3",
#                                         "weekday_4", "weekday_5", "weekday_6", "weekday_7"]
    
#     day_1_couponNumInEachMerPerUser = (df[colFor_couponNumInEachMerPerUser].groupby(["User_id", "Merchant_id"]).apply(lambda x: x[x["weekday_1"] == 1]["Coupon_id"].count())).to_frame().rename(columns = {0 : "day_1_couponNumInEachMerPerUser"}).reset_index()
#     df = pd.merge(df, day_1_couponNumInEachMerPerUser, how = "left", on = ["User_id", "Merchant_id"])
    
#     day_2_couponNumInEachMerPerUser = (df[colFor_couponNumInEachMerPerUser].groupby(["User_id", "Merchant_id"]).apply(lambda x: x[x["weekday_2"] == 1]["Coupon_id"].count())).to_frame().rename(columns = {0 : "day_2_couponNumInEachMerPerUser"}).reset_index()
#     df = pd.merge(df, day_2_couponNumInEachMerPerUser, how = "left", on = ["User_id", "Merchant_id"])
    
#     day_3_couponNumInEachMerPerUser = (df[colFor_couponNumInEachMerPerUser].groupby(["User_id", "Merchant_id"]).apply(lambda x: x[x["weekday_3"] == 1]["Coupon_id"].count())).to_frame().rename(columns = {0 : "day_3_couponNumInEachMerPerUser"}).reset_index()
#     df = pd.merge(df, day_3_couponNumInEachMerPerUser, how = "left", on = ["User_id", "Merchant_id"])
    
#     day_4_couponNumInEachMerPerUser = (df[colFor_couponNumInEachMerPerUser].groupby(["User_id", "Merchant_id"]).apply(lambda x: x[x["weekday_4"] == 1]["Coupon_id"].count())).to_frame().rename(columns = {0 : "day_4_couponNumInEachMerPerUser"}).reset_index()
#     df = pd.merge(df, day_4_couponNumInEachMerPerUser, how = "left", on = ["User_id", "Merchant_id"])
    
#     day_5_couponNumInEachMerPerUser = (df[colFor_couponNumInEachMerPerUser].groupby(["User_id", "Merchant_id"]).apply(lambda x: x[x["weekday_5"] == 1]["Coupon_id"].count())).to_frame().rename(columns = {0 : "day_5_couponNumInEachMerPerUser"}).reset_index()
#     df = pd.merge(df, day_5_couponNumInEachMerPerUser, how = "left", on = ["User_id", "Merchant_id"])
    
#     day_6_couponNumInEachMerPerUser = (df[colFor_couponNumInEachMerPerUser].groupby(["User_id", "Merchant_id"]).apply(lambda x: x[x["weekday_6"] == 1]["Coupon_id"].count())).to_frame().rename(columns = {0 : "day_6_couponNumInEachMerPerUser"}).reset_index()
#     df = pd.merge(df, day_6_couponNumInEachMerPerUser, how = "left", on = ["User_id", "Merchant_id"])
    
#     day_7_couponNumInEachMerPerUser = (df[colFor_couponNumInEachMerPerUser].groupby(["User_id", "Merchant_id"]).apply(lambda x: x[x["weekday_7"] == 1]["Coupon_id"].count())).to_frame().rename(columns = {0 : "day_7_couponNumInEachMerPerUser"}).reset_index()
#     df = pd.merge(df, day_7_couponNumInEachMerPerUser, how = "left", on = ["User_id", "Merchant_id"])

#     return df



# train = day_i_couponNumInEachMerPerUser(train)
# test = day_i_couponNumInEachMerPerUser(test)



In [23]:
# # 每位 user 在各 merchant 在 星期一到日 拿到的 coupon 數
# def day_2_couponNumInEachMerPerUser(df) :
    
#     day_2_couponNumInEachMerPerUser = (df.groupby(["User_id", "Merchant_id"]).apply(lambda x: x[x["weekday_2"] == 1]["Coupon_id"].count())).to_frame().rename(columns = {0 : "day_2_couponNumInEachMerPerUser"}).reset_index()
#     df = pd.merge(df, day_2_couponNumInEachMerPerUser, how = "left", on = ["User_id", "Merchant_id"])
#     return df

# # train = day_i_couponNumInEachMerPerUser(train)
# # test = day_i_couponNumInEachMerPerUser(test)

# day_2_couponNumInEachMerPerUser(test.head(100))

In [ ]:
# total_df['DiscountRatio'] = total_df.Discount_rate.map( lambda x: (1 - float(x.split(':')[1])/float(x.split(':')[0])) if(':' in x) else float(x) )


In [24]:
# 檢查 DataFrame 空缺值的狀態
def na_check(df_data):
    data_na = (df_data.isnull().sum() / len(df_data)) * 100
    data_na = data_na.drop(data_na[data_na == 0].index).sort_values(ascending=False)
    missing_data = pd.DataFrame({'Missing Ratio' :data_na})
    display(missing_data)

In [25]:
print("\n << na_check(train) >>")
display(na_check(train))

print("\n << na_check(test) >>")
display(na_check(test))


 << na_check(train) >>


,Missing Ratio
Date,94.25197


None


 << na_check(test) >>


,Missing Ratio


None

## Other Features

In [26]:
def extractFeature(dataset3):
    # this_month_user_receive_all_coupon_count
    t = dataset3[['User_id']]
    t['this_month_user_receive_all_coupon_count'] = 1
    t = t.groupby('User_id').agg('sum').reset_index()

    # this_month_user_receive_same_coupon_count
    t1 = dataset3[['User_id','Coupon_id']]
    t1['this_month_user_receive_same_coupon_count'] = 1
    t1 = t1.groupby(['User_id','Coupon_id']).agg('sum').reset_index()

    
    t2 = dataset3[['User_id','Coupon_id','Date_received']]
    t2.Date_received = t2.Date_received.astype('str')
    t2 = t2.groupby(['User_id','Coupon_id'])['Date_received'].agg(lambda x:':'.join(x)).reset_index()
    t2['receive_number'] = t2.Date_received.apply(lambda s:len(s.split(':')))
    t2 = t2[t2.receive_number>1]
    t2['max_Date_received'] = t2.Date_received.apply(lambda s:max([float(d) for d in s.split(':')]))
    t2['min_Date_received'] = t2.Date_received.apply(lambda s:min([float(d) for d in s.split(':')]))
    t2 = t2[['User_id','Coupon_id','max_Date_received','min_Date_received']]

    # this_month_user_receive_same_coupon_lastone, this_month_user_receive_same_coupon_firstone
    t3 = dataset3[['User_id','Coupon_id','Date_received']]
    t3 = pd.merge(t3,t2,on=['User_id','Coupon_id'],how='left')
    t3['this_month_user_receive_same_coupon_lastone'] = t3.max_Date_received - t3.Date_received
    t3['this_month_user_receive_same_coupon_firstone'] = t3.Date_received - t3.min_Date_received
    
    def is_firstlastone(x):
        if x==0:
            return 1
        elif x>0:
            return 0
        else:
            return -1 #those only receive once

    t3.this_month_user_receive_same_coupon_lastone = t3.this_month_user_receive_same_coupon_lastone.apply(is_firstlastone)
    t3.this_month_user_receive_same_coupon_firstone = t3.this_month_user_receive_same_coupon_firstone.apply(is_firstlastone)
    t3 = t3[['User_id','Coupon_id','Date_received','this_month_user_receive_same_coupon_lastone','this_month_user_receive_same_coupon_firstone']]

    t4 = dataset3[['User_id','Date_received']]
    t4['this_day_user_receive_all_coupon_count'] = 1
    t4 = t4.groupby(['User_id','Date_received']).agg('sum').reset_index()

    t5 = dataset3[['User_id','Coupon_id','Date_received']]
    t5['this_day_user_receive_same_coupon_count'] = 1
    t5 = t5.groupby(['User_id','Coupon_id','Date_received']).agg('sum').reset_index()

    t6 = dataset3[['User_id','Coupon_id','Date_received']]
    t6.Date_received = t6.Date_received.astype('str')
    t6 = t6.groupby(['User_id','Coupon_id'])['Date_received'].agg(lambda x:':'.join(x)).reset_index()
    t6.rename(columns={'Date_received':'dates'},inplace=True)

    def get_day_gap_before(s):
        if str(s)=="nan":
            return -1
        Date_received,dates = s.split('-')
        dates = dates.split(':')
        gaps = []
        for d in dates:
            this_gap = (date(int(Date_received[0:4]),int(Date_received[4:6]),int(Date_received[6:8]))-date(int(d[0:4]),int(d[4:6]),int(d[6:8]))).days
            if this_gap>0:
                gaps.append(this_gap)
        if len(gaps)==0:
            return -1
        else:
            return min(gaps)

    def get_day_gap_after(s):
        if str(s)=="nan":
            return -1
        Date_received,dates = s.split('-')
        dates = dates.split(':')
        gaps = []
        for d in dates:
            this_gap = (date(int(d[0:4]),int(d[4:6]),int(d[6:8]))-date(int(Date_received[0:4]),int(Date_received[4:6]),int(Date_received[6:8]))).days
            if this_gap>0:
                gaps.append(this_gap)
        if len(gaps)==0:
            return -1
        else:
            return min(gaps)


    t7 = dataset3[['User_id','Coupon_id','Date_received']]
    t7 = pd.merge(t7,t6,on=['User_id','Coupon_id'],how='left')
    t7['Date_received_date'] = t7.Date_received.astype('str') + '-' + t7.dates
    t7['day_gap_before'] = t7.Date_received_date.apply(get_day_gap_before)
    t7['day_gap_after'] = t7.Date_received_date.apply(get_day_gap_after)
    t7 = t7[['User_id','Coupon_id','Date_received','day_gap_before','day_gap_after']]

    other_feature3 = pd.merge(t1,t,on='User_id')
    other_feature3 = pd.merge(other_feature3,t3,on=['User_id','Coupon_id'])
    other_feature3 = pd.merge(other_feature3,t4,on=['User_id','Date_received'])
    other_feature3 = pd.merge(other_feature3,t5,on=['User_id','Coupon_id','Date_received'])
    other_feature3 = pd.merge(other_feature3,t7,on=['User_id','Coupon_id','Date_received'])
    #other_feature3.to_csv('data/other_feature3.csv',index=None)
    
    return other_feature3


In [27]:
train_moreFeatures = extractFeature(train)
test_moreFeatures = extractFeature(test)

# 將額外的feature併回去
train2 = pd.merge(train, train_moreFeatures, how = 'left', on = ['User_id', 'Coupon_id', 'Date_received'])
print("train2.head()", "\n")
display(train2.head())

test2 = pd.merge(test, test_moreFeatures, how = 'left', on = ['User_id', 'Coupon_id', 'Date_received'])
print("test2.head()", "\n")
display(test2.head())

train2.head() 



,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,label,weekday,weekday_type,...,day_6_couponNumInEachMerPerUser,day_7_couponNumInEachMerPerUser,this_month_user_receive_same_coupon_count,this_month_user_receive_all_coupon_count,this_month_user_receive_same_coupon_lastone,this_month_user_receive_same_coupon_firstone,this_day_user_receive_all_coupon_count,this_day_user_receive_same_coupon_count,day_gap_before,day_gap_after
0,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN,0,3.0,0,...,1,0,1,2,-1,-1,1,1,-1,-1
1,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN,0,6.0,0,...,1,0,1,2,-1,-1,1,1,-1,-1
2,1832624,3381,7610.0,200:20,0.0,20160429.0,NaN,0,5.0,0,...,0,0,1,1,-1,-1,1,1,-1,-1
3,2029232,3381,11951.0,200:20,1.0,20160129.0,NaN,0,5.0,0,...,0,0,1,1,-1,-1,1,1,-1,-1
4,2223968,3381,9776.0,10:5,2.0,20160129.0,NaN,0,5.0,0,...,0,0,1,1,-1,-1,1,1,-1,-1


test2.head() 



,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,weekday,weekday_type,weekday_1,weekday_2,...,day_6_couponNumInEachMerPerUser,day_7_couponNumInEachMerPerUser,this_month_user_receive_same_coupon_count,this_month_user_receive_all_coupon_count,this_month_user_receive_same_coupon_lastone,this_month_user_receive_same_coupon_firstone,this_day_user_receive_all_coupon_count,this_day_user_receive_same_coupon_count,day_gap_before,day_gap_after
0,1439408,4663,11002.0,150:20,1.0,20160528.0,6,0,0,0,...,1,0,1,3,-1,-1,1,1,-1,-1
1,1439408,2632,8591.0,20:1,0.0,20160613.0,1,0,1,0,...,0,0,2,3,1,0,1,1,28,-1
2,1439408,2632,8591.0,20:1,0.0,20160516.0,1,0,1,0,...,0,0,2,3,0,1,1,1,-1,28
3,2029232,450,1532.0,30:5,0.0,20160530.0,1,0,1,0,...,0,0,1,2,-1,-1,1,1,-1,-1
4,2029232,6459,12737.0,20:1,0.0,20160519.0,4,0,0,0,...,0,0,1,2,-1,-1,1,1,-1,-1


In [28]:
# drop duplicates 最後要預測的筆數才對，才能提交預測檔

train2 = train2.drop_duplicates().reset_index(drop = True)
test2 = test2.drop_duplicates().reset_index(drop = True)

# 因為之後的test2要取特徵的那些col而已，這邊先複製一份，作為提交檔取User_id, Coupon_id等等所用
train2_cp = train2.copy()
test2_cp = test2.copy()

In [29]:
## user - merchant  Feature

# def extract_user_merchant_Feature(feature3):
#     """
#     4.user_merchant:
#           times_user_buy_merchant_before. 
#     """
#     #for dataset3
#     all_user_merchant = feature3[['User_id','Merchant_id']]
#     all_user_merchant.drop_duplicates(inplace=True)

#     t = feature3[['User_id','Merchant_id','Date']]
#     t = t[t.Date!='null'][['User_id','Merchant_id']]
#     t['user_merchant_buy_total'] = 1
#     t = t.groupby(['User_id','Merchant_id']).agg('sum').reset_index()
#     t.drop_duplicates(inplace=True)

#     t1 = feature3[['User_id','Merchant_id','Coupon_id']]
#     t1 = t1[t1.Coupon_id!='null'][['User_id','Merchant_id']]
#     t1['user_merchant_received'] = 1
#     t1 = t1.groupby(['User_id','Merchant_id']).agg('sum').reset_index()
#     t1.drop_duplicates(inplace=True)

#     t2 = feature3[['User_id','Merchant_id','Date','Date_received']]
#     t2 = t2[(t2.Date!='null')&(t2.Date_received!='null')][['User_id','Merchant_id']]
#     t2['user_merchant_buy_use_coupon'] = 1
#     t2 = t2.groupby(['User_id','Merchant_id']).agg('sum').reset_index()
#     t2.drop_duplicates(inplace=True)

#     t3 = feature3[['User_id','Merchant_id']]
#     t3['user_merchant_any'] = 1
#     t3 = t3.groupby(['User_id','Merchant_id']).agg('sum').reset_index()
#     t3.drop_duplicates(inplace=True)

#     t4 = feature3[['User_id','Merchant_id','Date','Coupon_id']]
#     t4 = t4[(t4.Date!='null')&(t4.Coupon_id=='null')][['User_id','Merchant_id']]
#     t4['user_merchant_buy_common'] = 1
#     t4 = t4.groupby(['User_id','Merchant_id']).agg('sum').reset_index()
#     t4.drop_duplicates(inplace=True)

#     user_merchant3 = pd.merge(all_user_merchant,t,on=['User_id','Merchant_id'],how='left')
#     user_merchant3 = pd.merge(user_merchant3,t1,on=['User_id','Merchant_id'],how='left')
#     user_merchant3 = pd.merge(user_merchant3,t2,on=['User_id','Merchant_id'],how='left')
#     user_merchant3 = pd.merge(user_merchant3,t3,on=['User_id','Merchant_id'],how='left')
#     user_merchant3 = pd.merge(user_merchant3,t4,on=['User_id','Merchant_id'],how='left')
#     user_merchant3.user_merchant_buy_use_coupon = user_merchant3.user_merchant_buy_use_coupon.replace(np.nan,0)
#     user_merchant3.user_merchant_buy_common = user_merchant3.user_merchant_buy_common.replace(np.nan,0)
#     user_merchant3['user_merchant_coupon_transfer_rate'] = user_merchant3.user_merchant_buy_use_coupon.astype('float') / user_merchant3.user_merchant_received.astype('float')
#     user_merchant3['user_merchant_coupon_buy_rate'] = user_merchant3.user_merchant_buy_use_coupon.astype('float') / user_merchant3.user_merchant_buy_total.astype('float')
#     user_merchant3['user_merchant_rate'] = user_merchant3.user_merchant_buy_total.astype('float') / user_merchant3.user_merchant_any.astype('float')
#     user_merchant3['user_merchant_common_buy_rate'] = user_merchant3.user_merchant_buy_common.astype('float') / user_merchant3.user_merchant_buy_total.astype('float')
    
#     return user_merchant3
    

    
    
    
# train_user_merchant_Feature = extract_user_merchant_Feature(train)
# test_user_merchant_Feature = extract_user_merchant_Feature(test)

# # 將額外的feature併回去
# train3 = pd.merge(train2, train_user_merchant_Feature, how = 'left', on = ['User_id', 'Merchant_id'])
# print("train3.head()", "\n")
# display(train3.head())

# test3 = pd.merge(test2, test_user_merchant_Feature, how = 'left', on = ['User_id', 'Merchant_id'])
# print("test3.head()", "\n")
# display(test3.head())



# # drop duplicates 最後要預測的筆數才對，才能提交預測檔

# train3 = train3.drop_duplicates().reset_index(drop = True)
# test3 = test3.drop_duplicates().reset_index(drop = True)

# # 因為之後的test3要取特徵的那些col而已，這邊先複製一份，作為提交檔取User_id, Coupon_id等等所用
# train3_cp = train3.copy()
# test3_cp = test3.copy()



print("test 沒有 Date(消費日期) 這個欄位，就無法衍生出label這個欄位，所以不能照這個想法做")

test 沒有 Date(消費日期) 這個欄位，就無法衍生出label這個欄位，所以不能照這個想法做


In [30]:
train.columns

Index(['User_id', 'Merchant_id', 'Coupon_id', 'Discount_rate', 'Distance',
       'Date_received', 'Date', 'label', 'weekday', 'weekday_type',
       'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5',
       'weekday_6', 'weekday_7', 'discount_rate', 'discount_man',
       'discount_jian', 'discount_type', 'merchantNumPerUser', 'distPerUser',
       'couponNumPerUser', 'goodDiscountCouNumPerUser',
       'goodDiscountrateAvgDistPerUser', 'couponNumInEachMerPerUser',
       'NUniqueMerNumPerUser', 'day_1_couponNumInEachMerPerUser',
       'day_2_couponNumInEachMerPerUser', 'day_3_couponNumInEachMerPerUser',
       'day_4_couponNumInEachMerPerUser', 'day_5_couponNumInEachMerPerUser',
       'day_6_couponNumInEachMerPerUser', 'day_7_couponNumInEachMerPerUser'],
      dtype='object')

In [31]:
train2.columns

Index(['User_id', 'Merchant_id', 'Coupon_id', 'Discount_rate', 'Distance',
       'Date_received', 'Date', 'label', 'weekday', 'weekday_type',
       'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5',
       'weekday_6', 'weekday_7', 'discount_rate', 'discount_man',
       'discount_jian', 'discount_type', 'merchantNumPerUser', 'distPerUser',
       'couponNumPerUser', 'goodDiscountCouNumPerUser',
       'goodDiscountrateAvgDistPerUser', 'couponNumInEachMerPerUser',
       'NUniqueMerNumPerUser', 'day_1_couponNumInEachMerPerUser',
       'day_2_couponNumInEachMerPerUser', 'day_3_couponNumInEachMerPerUser',
       'day_4_couponNumInEachMerPerUser', 'day_5_couponNumInEachMerPerUser',
       'day_6_couponNumInEachMerPerUser', 'day_7_couponNumInEachMerPerUser',
       'this_month_user_receive_same_coupon_count',
       'this_month_user_receive_all_coupon_count',
       'this_month_user_receive_same_coupon_lastone',
       'this_month_user_receive_same_coupon_firstone',
      

In [32]:
train2[["User_id", "Merchant_id", "Coupon_id",
       'this_month_user_receive_same_coupon_count',
       'this_month_user_receive_all_coupon_count',
       'this_month_user_receive_same_coupon_lastone',
       'this_month_user_receive_same_coupon_firstone',
       'this_day_user_receive_all_coupon_count',
       'this_day_user_receive_same_coupon_count', 'day_gap_before',
       'day_gap_after',
       "couponNumInEachMerPerUser", "NUniqueMerNumPerUser"]].iloc[0:20, :]

,User_id,Merchant_id,Coupon_id,this_month_user_receive_same_coupon_count,this_month_user_receive_all_coupon_count,this_month_user_receive_same_coupon_lastone,this_month_user_receive_same_coupon_firstone,this_day_user_receive_all_coupon_count,this_day_user_receive_same_coupon_count,day_gap_before,day_gap_after,couponNumInEachMerPerUser,NUniqueMerNumPerUser
0,1439408,2632,8591.0,1,2,-1,-1,1,1,-1,-1,2,1
1,1439408,2632,1078.0,1,2,-1,-1,1,1,-1,-1,2,1
2,1832624,3381,7610.0,1,1,-1,-1,1,1,-1,-1,1,1
3,2029232,3381,11951.0,1,1,-1,-1,1,1,-1,-1,1,1
4,2223968,3381,9776.0,1,1,-1,-1,1,1,-1,-1,1,1
5,73611,2099,12034.0,1,1,-1,-1,1,1,-1,-1,1,1
6,163606,1569,5054.0,1,1,-1,-1,1,1,-1,-1,1,1
7,3273056,4833,7802.0,1,1,-1,-1,1,1,-1,-1,1,1
8,94107,3381,7610.0,1,1,-1,-1,1,1,-1,-1,1,1
9,253750,8390,7531.0,1,1,-1,-1,1,1,-1,-1,1,1


In [33]:
col = ['discount_rate', 'discount_type', 'discount_man', 'discount_jian', 
       'Distance', 
       'weekday', 'weekday_type', 
       'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'weekday_7',
       'merchantNumPerUser',
       'distPerUser',
       'couponNumPerUser',
       'goodDiscountCouNumPerUser',
       'goodDiscountrateAvgDistPerUser'] + ['this_month_user_receive_same_coupon_count',
       'this_month_user_receive_all_coupon_count',
       'this_month_user_receive_same_coupon_lastone',
       'this_month_user_receive_same_coupon_firstone',
       'this_day_user_receive_all_coupon_count',
       'this_day_user_receive_same_coupon_count', 'day_gap_before',
       'day_gap_after'] + ["couponNumInEachMerPerUser", "NUniqueMerNumPerUser"] + ["day_1_couponNumInEachMerPerUser",
       "day_2_couponNumInEachMerPerUser", "day_3_couponNumInEachMerPerUser", "day_4_couponNumInEachMerPerUser",
       "day_5_couponNumInEachMerPerUser", "day_6_couponNumInEachMerPerUser", "day_7_couponNumInEachMerPerUser"]

In [44]:
# col = ['discount_rate', 'discount_type', 'discount_man', 'discount_jian', 
#        'Distance', 
#        'weekday_type', 
#        'merchantNumPerUser',
#        'distPerUser',
#        'couponNumPerUser',
#        'goodDiscountCouNumPerUser',
#        'goodDiscountrateAvgDistPerUser'] + ['this_month_user_receive_same_coupon_count',
#        'this_month_user_receive_all_coupon_count',
#        'this_month_user_receive_same_coupon_lastone',
#        'this_month_user_receive_same_coupon_firstone',
#        'this_day_user_receive_all_coupon_count',
#        'this_day_user_receive_same_coupon_count', 'day_gap_before',
#        'day_gap_after'] + ["couponNumInEachMerPerUser", "NUniqueMerNumPerUser"] + ["day_1_couponNumInEachMerPerUser",
#        "day_2_couponNumInEachMerPerUser", "day_3_couponNumInEachMerPerUser", "day_4_couponNumInEachMerPerUser",
#        "day_5_couponNumInEachMerPerUser", "day_6_couponNumInEachMerPerUser", "day_7_couponNumInEachMerPerUser"]

In [34]:
train2[col].head()

,discount_rate,discount_type,discount_man,discount_jian,Distance,weekday,weekday_type,weekday_1,weekday_2,weekday_3,...,day_gap_after,couponNumInEachMerPerUser,NUniqueMerNumPerUser,day_1_couponNumInEachMerPerUser,day_2_couponNumInEachMerPerUser,day_3_couponNumInEachMerPerUser,day_4_couponNumInEachMerPerUser,day_5_couponNumInEachMerPerUser,day_6_couponNumInEachMerPerUser,day_7_couponNumInEachMerPerUser
0,0.95,1,20,1,0.0,3.0,0,0,0,1,...,-1,2,1,0,0,1,0,0,1,0
1,0.95,1,20,1,0.0,6.0,0,0,0,0,...,-1,2,1,0,0,1,0,0,1,0
2,0.90,1,200,20,0.0,5.0,0,0,0,0,...,-1,1,1,0,0,0,0,1,0,0
3,0.90,1,200,20,1.0,5.0,0,0,0,0,...,-1,1,1,0,0,0,0,1,0,0
4,0.50,1,10,5,2.0,5.0,0,0,0,0,...,-1,1,1,0,0,0,0,1,0,0


In [35]:
print("\n << na_check(train2[col]) >>")
display(na_check(train2[col]))

print("\n << na_check(test2[col]) >>")
display(na_check(test2[col]))


 << na_check(train2[col]) >>


,Missing Ratio


None


 << na_check(test2[col]) >>


,Missing Ratio


None

In [45]:
# MinMaxScaler
X = train2[col]
y = np.ravel(train2["label"])

# mms = MinMaxScaler()
# X = mms.fit_transform(train2[col])

# 切分訓練集/測試集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 87)

# 哭喔 剛剛train-test-split 的 X_test撞名稱了啦 下面這個才是我要的
# test2 = mms.fit_transform(test2[col])
test2 = test2[col]

In [46]:
# 用第二版的資料整理而已
#----------------------------------------------------------- max_features 也改成 10
lr = LogisticRegression(tol=0.001, penalty='l2', fit_intercept=True, C=1.0)

gdbt = GradientBoostingClassifier(tol=100, subsample=0.75, n_estimators=250, max_features=10, # 最多要看col有幾個feature
                                  max_depth=6, learning_rate=0.03)

rf = RandomForestClassifier(n_estimators=100, min_samples_split=2, min_samples_leaf=1, 
                            max_features='sqrt', max_depth=6, bootstrap=True,
                            random_state = 87)

In [38]:
len(pd.read_csv(os.path.join(data_path, 'midTerm_1.csv')))

304096

In [47]:
# 線性迴歸預測檔 (結果有部分隨機, 請以 Kaggle 計算的得分為準, 以下模型同理)
lr.fit(X_train, y_train)
lr_pred = lr.predict_proba(test2)[:,1]

In [ ]:
# 梯度提升機預測檔 
gdbt.fit(X_train, y_train)
gdbt_pred = gdbt.predict_proba(test2)[:,1]

In [48]:
# 隨機森林預測檔
rf.fit(X_train, y_train)
rf_pred = rf.predict_proba(test2)[:,1]

## 加入 xgboost

In [49]:
from xgboost import XGBClassifier

In [50]:
# 建立模型
xgb = XGBClassifier(max_depth=3, n_estimators=300, 
                    learning_rate=0.05, random_state = 87)
# xgboost 預測檔
xgb.fit(X_train, y_train)
xgb_pred = xgb.predict_proba(test2)[:,1]

In [ ]:
# # submit
# def createUid(row):
#     return str(row['User_id']) + '_' + str(int(row['Coupon_id'])) + '_' + str(int(row['Date_received']))

# submit = pd.DataFrame()
# submit['uid'] = test2_cp.apply(createUid, axis=1)
# predict = lr_pred * 0.12 + rf_pred * 0.8 + xgb_pred * 0.08
# submit['label'] = predict

# submit = submit.groupby("uid", as_index=False).mean()

# # submit.to_csv(os.path.join(data_path, 'midTerm_22.csv'), index=False)


# # score : 0.77685

In [ ]:
# # submit
# def createUid(row):
#     return str(row['User_id']) + '_' + str(int(row['Coupon_id'])) + '_' + str(int(row['Date_received']))

# submit = pd.DataFrame()
# submit['uid'] = test2_cp.apply(createUid, axis=1)
# predict = lr_pred * 0.2 + rf_pred * 0.7 + xgb_pred * 0.1
# submit['label'] = predict

# submit = submit.groupby("uid", as_index=False).mean()

# # submit.to_csv(os.path.join(data_path, 'midTerm_23.csv'), index=False)


# # score : 0.77731

In [ ]:
# # submit
# def createUid(row):
#     return str(row['User_id']) + '_' + str(int(row['Coupon_id'])) + '_' + str(int(row['Date_received']))

# submit = pd.DataFrame()
# submit['uid'] = test2_cp.apply(createUid, axis=1)
# predict = lr_pred * 0.2 + rf_pred * 0.6 + xgb_pred * 0.2
# submit['label'] = predict

# submit = submit.groupby("uid", as_index=False).mean()

# submit.to_csv(os.path.join(data_path, 'midTerm_24.csv'), index=False)


# # score : 0.77695

In [ ]:
# # submit
# def createUid(row):
#     return str(row['User_id']) + '_' + str(int(row['Coupon_id'])) + '_' + str(int(row['Date_received']))

# submit = pd.DataFrame()
# submit['uid'] = test2_cp.apply(createUid, axis=1)
# predict = lr_pred * 0.2 + rf_pred * 0.1 + xgb_pred * 0.7
# submit['label'] = predict

# submit = submit.groupby("uid", as_index=False).mean()

# submit.to_csv(os.path.join(data_path, 'midTerm_25.csv'), index=False)


# # score : 0.77137

In [ ]:
# # submit
# def createUid(row):
#     return str(row['User_id']) + '_' + str(int(row['Coupon_id'])) + '_' + str(int(row['Date_received']))

# submit = pd.DataFrame()
# submit['uid'] = test2_cp.apply(createUid, axis=1)
# predict = lr_pred * 0.15 + rf_pred * 0.8 + xgb_pred * 0.05
# submit['label'] = predict

# submit = submit.groupby("uid", as_index=False).mean()

# # submit.to_csv(os.path.join(data_path, 'midTerm_26.csv'), index=False)


# # score : 0.77699

In [ ]:
# submit
def createUid(row):
    return str(row['User_id']) + '_' + str(int(row['Coupon_id'])) + '_' + str(int(row['Date_received']))

submit = pd.DataFrame()
submit['uid'] = test2_cp.apply(createUid, axis=1)
predict = lr_pred * 0.3 + rf_pred * 0.6 + xgb_pred * 0.1
submit['label'] = predict

submit = submit.groupby("uid", as_index=False).mean()

# submit.to_csv(os.path.join(data_path, 'midTerm_27.csv'), index=False)


# Public Score : 0.77765

In [ ]:
# # submit
# def createUid(row):
#     return str(row['User_id']) + '_' + str(int(row['Coupon_id'])) + '_' + str(int(row['Date_received']))

# submit = pd.DataFrame()
# submit['uid'] = test2_cp.apply(createUid, axis=1)
# predict = lr_pred * 0.4 + rf_pred * 0.5 + xgb_pred * 0.1
# submit['label'] = predict

# submit = submit.groupby("uid", as_index=False).mean()

# # submit.to_csv(os.path.join(data_path, 'midTerm_28.csv'), index=False)


# # score : 0.77759

In [ ]:
# submit
def createUid(row):
    return str(row['User_id']) + '_' + str(int(row['Coupon_id'])) + '_' + str(int(row['Date_received']))

submit = pd.DataFrame()
submit['uid'] = test2_cp.apply(createUid, axis=1)
predict = lr_pred * 0.25 + rf_pred * 0.7 + xgb_pred * 0.05
submit['label'] = predict

submit = submit.groupby("uid", as_index=False).mean()

submit.to_csv(os.path.join(data_path, 'midTerm_29.csv'), index=False)


# Public Score : 0.77745

In [ ]:
# # submit
# def createUid(row):
#     return str(row['User_id']) + '_' + str(int(row['Coupon_id'])) + '_' + str(int(row['Date_received']))

# submit = pd.DataFrame()
# submit['uid'] = test2_cp.apply(createUid, axis=1)
# predict = lr_pred * 0.7 + rf_pred * 0.2 + xgb_pred * 0.1
# submit['label'] = predict

# submit = submit.groupby("uid", as_index=False).mean()

# submit.to_csv(os.path.join(data_path, 'midTerm_30.csv'), index=False)


# # score : 0.77369

## Stacking

In [ ]:
# from mlxtend.classifier import StackingClassifier

# meta_estimator = GradientBoostingClassifier(tol=100, subsample=0.70, n_estimators=100, 
#                                            max_features='sqrt', max_depth=4, learning_rate=0.3)
# """
# Your Code Here
# """
# stacking = StackingClassifier(classifiers=[lr, rf, xgb], meta_classifier = meta_estimator)


# stacking.fit(X_train, y_train)
# stacking_pred = stacking.predict(test2)


# # submit
# def createUid(row):
#     return str(row['User_id']) + '_' + str(int(row['Coupon_id'])) + '_' + str(int(row['Date_received']))

# submit = pd.DataFrame()
# submit['uid'] = test2_cp.apply(createUid, axis=1)
# predict = stacking_pred
# submit['label'] = predict

# submit = submit.groupby("uid", as_index=False).mean()

# # submit.to_csv(os.path.join(data_path, 'midTerm_31.csv'), index=False)


# # Public Score : 0.53208

In [ ]:
# from mlxtend.classifier import StackingClassifier

# meta_estimator = RandomForestClassifier(n_estimators=200, min_samples_split=2, min_samples_leaf=2, 
#                             max_features='sqrt', max_depth=5, bootstrap=True,
#                             random_state = 87)
# """
# Your Code Here
# """
# stacking = StackingClassifier(classifiers=[lr, rf, xgb], meta_classifier = meta_estimator)


# stacking.fit(X_train, y_train)
# stacking_pred = stacking.predict(test2)


# # submit
# def createUid(row):
#     return str(row['User_id']) + '_' + str(int(row['Coupon_id'])) + '_' + str(int(row['Date_received']))

# submit = pd.DataFrame()
# submit['uid'] = test2_cp.apply(createUid, axis=1)
# predict = stacking_pred
# submit['label'] = predict

# submit = submit.groupby("uid", as_index=False).mean()

# submit.to_csv(os.path.join(data_path, 'midTerm_32.csv'), index=False)


# # Public Score : 0.53202

## 加入新特徵後回到Blending

In [43]:
# submit
def createUid(row):
    return str(row['User_id']) + '_' + str(int(row['Coupon_id'])) + '_' + str(int(row['Date_received']))

submit = pd.DataFrame()
submit['uid'] = test2_cp.apply(createUid, axis=1)
predict = lr_pred * 0.3 + rf_pred * 0.6 + xgb_pred * 0.1
submit['label'] = predict

submit = submit.groupby("uid", as_index=False).mean()

# submit.to_csv(os.path.join(data_path, 'midTerm_33.csv'), index=False)


# Public Score : 0.77668

In [51]:
# submit
def createUid(row):
    return str(row['User_id']) + '_' + str(int(row['Coupon_id'])) + '_' + str(int(row['Date_received']))

submit = pd.DataFrame()
submit['uid'] = test2_cp.apply(createUid, axis=1)
predict = lr_pred * 0.3 + rf_pred * 0.6 + xgb_pred * 0.1
submit['label'] = predict

submit = submit.groupby("uid", as_index=False).mean()

# submit.to_csv(os.path.join(data_path, 'midTerm_34.csv'), index=False)


# Public Score : 0.77664

https://github.com/ben19770209 厲害的人

https://nbviewer.jupyter.org/github/tobby168/100Day-ML-Marathon/blob/master/Midterm.ipynb

https://www.kaggle.com/lianglirong/alibaba-tianchi-o2o kaggle kernel


https://tianchi.aliyun.com/competition/entrance/231593/introduction 官方網站

https://redstonewill.com/1681/ 紅色石頭: 介紹與範例

https://github.com/RedstoneWill/MachineLearningInAction-Camp/blob/master/Week1/o2o-1.ipynb (0.532344469452)

https://github.com/RedstoneWill/MachineLearningInAction-Camp/blob/master/Week5/Assignments/o2o-1.ipynb (0.532344469452)

https://github.com/RedstoneWill/MachineLearningInAction-Camp/blob/master/Week10/Assignments/o2o-2.ipynb (0.630400363653)

https://github.com/wepe/O2O-Coupon-Usage-Forecast 第一名隊伍的註解

https://www.kaggle.com/lianglirong/alibaba-tianchi-o2o (0.9094526220772331) 範例

https://tianchi.aliyun.com/notebook-ai/detail?spm=5176.12586969.1002.15.29281b48IrTtER&postId=8462 (0.909452622077) 100行代码入门天池O2O优惠券使用新人赛【精简教程版】

https://www.kaggle.com/cbrogan/xgboost-example-python XGBoost example (Python)

https://github.com/wepe/O2O-Coupon-Usage-Forecast

https://github.com/wepe/O2O-Coupon-Usage-Forecast/blob/master/code/wepon/season%20one/extract_feature.py

In [ ]:
import warnings 
warnings.filterwarnings('ignore')

In [ ]:
def extractFeature(dataset3):
    # this_month_user_receive_all_coupon_count
    t = dataset3[['User_id']]
    t['this_month_user_receive_all_coupon_count'] = 1
    t = t.groupby('User_id').agg('sum').reset_index()
    print("\n t : this_month_user_receive_all_coupon_count")
    display(t.head())

    # this_month_user_receive_same_coupon_count
    t1 = dataset3[['User_id','Coupon_id']]
    t1['this_month_user_receive_same_coupon_count'] = 1
    t1 = t1.groupby(['User_id','Coupon_id']).agg('sum').reset_index()
    print("\n t1 : this_month_user_receive_same_coupon_count")
    display(t1.head())

    
    t2 = dataset3[['User_id','Coupon_id','Date_received']]
    t2.Date_received = t2.Date_received.astype('str')
    t2 = t2.groupby(['User_id','Coupon_id'])['Date_received'].agg(lambda x:':'.join(x)).reset_index()
    t2['receive_number'] = t2.Date_received.apply(lambda s:len(s.split(':')))
    t2 = t2[t2.receive_number>1]
    t2['max_Date_received'] = t2.Date_received.apply(lambda s:max([float(d) for d in s.split(':')]))
    t2['min_Date_received'] = t2.Date_received.apply(lambda s:min([float(d) for d in s.split(':')]))
    t2 = t2[['User_id','Coupon_id','max_Date_received','min_Date_received']]
    print("\n t2 : max_Date_received, min_Date_received")
    display(t2.head())

    # this_month_user_receive_same_coupon_lastone, this_month_user_receive_same_coupon_firstone
    t3 = dataset3[['User_id','Coupon_id','Date_received']]
    t3 = pd.merge(t3,t2,on=['User_id','Coupon_id'],how='left')
    t3['this_month_user_receive_same_coupon_lastone'] = t3.max_Date_received - t3.Date_received
    t3['this_month_user_receive_same_coupon_firstone'] = t3.Date_received - t3.min_Date_received
    print("\n t3 : this_month_user_receive_same_coupon_lastone, this_month_user_receive_same_coupon_firstone")
    display(t3.head())
    
    def is_firstlastone(x):
        if x==0:
            return 1
        elif x>0:
            return 0
        else:
            return -1 #those only receive once

    t3.this_month_user_receive_same_coupon_lastone = t3.this_month_user_receive_same_coupon_lastone.apply(is_firstlastone)
    t3.this_month_user_receive_same_coupon_firstone = t3.this_month_user_receive_same_coupon_firstone.apply(is_firstlastone)
    t3 = t3[['User_id','Coupon_id','Date_received','this_month_user_receive_same_coupon_lastone','this_month_user_receive_same_coupon_firstone']]
    print("\n t3 : this_month_user_receive_same_coupon_lastone, this_month_user_receive_same_coupon_firstone")
    display(t3.head())
    
    # this_day_user_receive_all_coupon_count
    t4 = dataset3[['User_id','Date_received']]
    t4['this_day_user_receive_all_coupon_count'] = 1
    t4 = t4.groupby(['User_id','Date_received']).agg('sum').reset_index()
    print("\n t4 : this_day_user_receive_all_coupon_count")
    display(t4.head())
    
    # Date_received
    t5 = dataset3[['User_id','Coupon_id','Date_received']]
    t5['this_day_user_receive_same_coupon_count'] = 1
    t5 = t5.groupby(['User_id','Coupon_id','Date_received']).agg('sum').reset_index()
    print("\n t5 : Date_received")
    display(t5.head())
    
    t6 = dataset3[['User_id','Coupon_id','Date_received']]
    t6.Date_received = t6.Date_received.astype('str')
    t6 = t6.groupby(['User_id','Coupon_id'])['Date_received'].agg(lambda x:':'.join(x)).reset_index()
    t6.rename(columns={'Date_received':'dates'},inplace=True)
    print("\n t6 : rename(columns={'Date_received':'dates'},inplace=True)")
    display(t6.head())
    
    def get_day_gap_before(s):
        if str(s)=="nan":
            return -1
        Date_received,dates = s.split('-')
        dates = dates.split(':')
        gaps = []
        for d in dates:
            this_gap = (date(int(Date_received[0:4]),int(Date_received[4:6]),int(Date_received[6:8]))-date(int(d[0:4]),int(d[4:6]),int(d[6:8]))).days
            if this_gap>0:
                gaps.append(this_gap)
        if len(gaps)==0:
            return -1
        else:
            return min(gaps)

    def get_day_gap_after(s):
        if str(s)=="nan":
            return -1
        Date_received,dates = s.split('-')
        dates = dates.split(':')
        gaps = []
        for d in dates:
            this_gap = (date(int(d[0:4]),int(d[4:6]),int(d[6:8]))-date(int(Date_received[0:4]),int(Date_received[4:6]),int(Date_received[6:8]))).days
            if this_gap>0:
                gaps.append(this_gap)
        if len(gaps)==0:
            return -1
        else:
            return min(gaps)


    t7 = dataset3[['User_id','Coupon_id','Date_received']]
    t7 = pd.merge(t7,t6,on=['User_id','Coupon_id'],how='left')
    t7['Date_received_date'] = t7.Date_received.astype('str') + '-' + t7.dates
    t7['day_gap_before'] = t7.Date_received_date.apply(get_day_gap_before)
    t7['day_gap_after'] = t7.Date_received_date.apply(get_day_gap_after)
    t7 = t7[['User_id','Coupon_id','Date_received','day_gap_before','day_gap_after']]
    print("\n t7 : day_gap_before, day_gap_after")
    display(t7.head())
    
    other_feature3 = pd.merge(t1,t,on='User_id')
    other_feature3 = pd.merge(other_feature3,t3,on=['User_id','Coupon_id'])
    other_feature3 = pd.merge(other_feature3,t4,on=['User_id','Date_received'])
    other_feature3 = pd.merge(other_feature3,t5,on=['User_id','Coupon_id','Date_received'])
    other_feature3 = pd.merge(other_feature3,t7,on=['User_id','Coupon_id','Date_received'])
    #other_feature3.to_csv('data/other_feature3.csv',index=None)
    
    return other_feature3


extractFeature(train).iloc[0:50, :]